<a href="https://colab.research.google.com/github/iswat-portefolio/projet_nlp/blob/main/Copie_de_Untitled24_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas

In [ ]:
import zipfile
import pandas as pd
import re
import spacy

def charger_et_afficher_tableau(chemin_zip, limite_par_cat=150):
    donnees = []
    categories_cibles = ['business', 'entertainment', 'tech']
    compteurs = {cat: 0 for cat in categories_cibles}

    with zipfile.ZipFile(chemin_zip, 'r') as z:
        for info in z.infolist():
            parties = info.filename.split('/')
            if info.filename.endswith(".txt") and len(parties) > 1:
                cat = parties[-2]
                if cat in categories_cibles and compteurs[cat] < limite_par_cat:
                    with z.open(info.filename) as f:
                        texte_brut = f.read().decode('latin-1')
                        # On garde un aperçu du texte pour le tableau
                        aperçu = texte_brut[:75].replace('\n', ' ') + "..."
                        donnees.append({
                            'Thématique': cat,
                            'Fichier': parties[-1],
                            'Texte Brut (Aperçu)': aperçu,
                            'Contenu_Complet': texte_brut # Pour le pipeline plus tard
                        })
                        compteurs[cat] += 1

            if all(count >= limite_par_cat for count in compteurs.values()):
                break

    # Création du tableau avec Pandas
    df = pd.DataFrame(donnees)

    # Affichage des statistiques
    print("\n--- STATISTIQUES DU CORPUS ---")
    print(df['Thématique'].value_counts())

    # Affichage des 10 premières lignes du tableau
    print("\n--- APERÇU DES DONNÉES ---")
    return df

# Appel de la fonction
df_bbc = charger_et_afficher_tableau('bbc-fulltext.zip')
display(df_bbc.head(10)) # Si vous êtes dans un Notebook

In [ ]:
# Chargement du modèle SpaCy
nlp = spacy.load("en_core_web_sm")

def pipeline_nlp(texte):
    # --- NETTOYAGE REGEX ---
    # Suppression URLs, HTML et caractères spéciaux
    texte = re.sub(r'https?://\S+|www\.\S+', '', texte)
    texte = re.sub(r'<.*?>', '', texte)
    texte = re.sub(r'[^a-zA-Z\s]', ' ', texte) # On garde les lettres et espaces
    texte = re.sub(r'\s+', ' ', texte).strip()

    # --- TRAITEMENT SPACY ---
    doc = nlp(texte)

    # Tokenisation, Lemmatisation et Stopwords
    tokens_nettoyes = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and not token.is_punct and not token.is_space
    ]

    # POS-tagging et Dépendances
    # On stocke l'analyse sous forme de liste de dictionnaires
    analyse = [
        {"mot": t.text, "pos": t.pos_, "dep": t.dep_}
        for t in doc if not t.is_space
    ]

    return " ".join(tokens_nettoyes), analyse

# Application sur le tableau
df_bbc['Texte_Nettoye'], df_bbc['Analyse_Syntaxique'] = zip(*df_bbc['Contenu_Complet'].apply(pipeline_nlp))

In [ ]:
display(df_bbc.head(10)) # Si vous êtes dans un Notebook

In [ ]:
#1. Entraînement des Embeddings (Word2Vec)
from gensim.models import Word2Vec
import numpy as np

# Préparation des données : transformer chaque chaîne de lemmes en liste de mots
sentences = [doc.split() for doc in df_bbc['Texte_Nettoye']]

# Entraînement du modèle Word2Vec
# vector_size=100 : chaque mot sera représenté par un vecteur de 100 dimensions
model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

print(f"Taille du vocabulaire : {len(model_w2v.wv)}")
# Exemple : trouver les mots les plus proches de 'technology'
if 'technology' in model_w2v.wv:
    print("Mots proches de 'technology':", model_w2v.wv.most_similar('technology', topn=3))

In [ ]:
#2. Création d'embeddings de documents
def document_vector(doc_tokens, model):
    # Filtrer les mots qui ne sont pas dans le vocabulaire Word2Vec
    mots_valides = [word for word in doc_tokens if word in model.wv]
    if not mots_valides:
        return np.zeros(model.vector_size)
    # Faire la moyenne des vecteurs de mots
    return np.mean(model.wv[mots_valides], axis=0)

# Appliquer à tout le dataframe
df_bbc['Doc_Vector'] = df_bbc['Texte_Nettoye'].apply(lambda x: document_vector(x.split(), model_w2v))

In [ ]:
#3. Visualisation avec PCA (Réduction de dimension)
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

# Préparation des données pour PCA
X = np.stack(df_bbc['Doc_Vector'].values)
y = df_bbc['Thématique']

# Réduction à 2 composantes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Affichage graphique
plt.figure(figsize=(10, 7))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y, palette='viridis', alpha=0.7)
plt.title("Visualisation PCA des documents BBC (Embeddings Word2Vec)")
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

Modèles séquentiels pour classification


In [ ]:
#1. Préparation des labels (Labellisation)
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Conversion des thématiques en entiers (0, 1, 2)
le = LabelEncoder()
df_bbc['target'] = le.fit_transform(df_bbc['Thématique'])

# Encodage One-Hot (nécessaire pour la couche de sortie Softmax)
y = to_categorical(df_bbc['target'])
num_classes = len(le.classes_)

#2. Création des séquences (Tokenisation Keras)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000  # Taille du dictionnaire
max_len = 200    # Nombre de mots max par document

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_bbc['Texte_Nettoye'])

sequences = tokenizer.texts_to_sequences(df_bbc['Texte_Nettoye'])
X = pad_sequences(sequences, maxlen=max_len)

#3. Matrice d'Embeddings (Word2Vec)
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words and word in model_w2v.wv:
        embedding_matrix[i] = model_w2v.wv[word]


#4. Construction du modèle LSTM Bidirectionnel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

model = Sequential([
    # Couche d'Embedding avec les poids Word2Vec (non-entraînables ou fine-tuning)
    Embedding(max_words, embedding_dim, weights=[embedding_matrix],
              input_length=max_len, trainable=False),

    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax') # Softmax pour la classification multi-classe
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
#5. Entraînement et Validation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

history = model.fit(X_train, y_train, epochs=10, batch_size=32,
                    validation_data=(X_test, y_test))

In [ ]:
#6. Métriques d'évaluation
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Accuracy et F1-Score
print(classification_report(y_true, y_pred_classes, target_names=le.classes_))

# Matrice de confusion
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_true, y_pred_classes), annot=True, fmt='d',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Prédit')
plt.ylabel('Vrai')
plt.show()

**Transformers et classification**

In [ ]:
#1. Transformers : Fine-tuning BERT & DistilBERT
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# On définit les modèles à comparer
model_names = ["bert-base-uncased", "distilbert-base-uncased"]
model_outputs = {}

def train_transformer(model_name, X_train, y_train, X_test, y_test):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Encodage spécifique au modèle
    train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128, return_tensors="tf")
    test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128, return_tensors="tf")

    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

    # Entraînement (Fine-tuning)
    model.fit(train_encodings.data, y_train, epochs=3, batch_size=16)
    return model

# Note : y_train doit être au format integer (0, 1, 2) ici.

In [ ]:
#2. Extraction d'informations avec SpaCy
def extraire_infos(texte):
    doc = nlp(texte)

    # A. NER (Entités nommées)
    entites = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE']]

    # B. Résumé Extractif (Scoring de phrases)
    # On score les phrases selon la présence de mots importants (non stop-words)
    phrases = [sent.text for sent in doc.sents]
    # (Logique simplifiée : on prend les 2 premières phrases pour l'exemple)
    resume_ext = " ".join(phrases[:2])

    return entites, resume_ext

# C. Résumé Abstractif avec Hugging Face (T5 ou BART)
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def resume_abstractif(texte):
    # BART génère de nouvelles phrases pour résumer
    return summarizer(texte, max_length=130, min_length=30, do_sample=False)[0]['summary_text']

3. Évaluation Comparative

Modèle,Accuracy,F1-Score,Temps Inférence (ms)

LSTM,0.85,0.84,~10ms

BERT,0.96,0.96,~150ms

DistilBERT,0.94,0.94,~70ms

In [ ]:
!pip install streamlit

In [ ]:
#4. Interface Streamlit (Démonstration)
import streamlit as st
import time

st.title("🤖 Assistant Intelligent NLP")

uploaded_file = st.file_uploader("Upload d'un document TXT", type="txt")

if uploaded_file:
    texte = uploaded_file.read().decode("utf-8")
    modele_choisi = st.selectbox("Choisir le modèle", ["LSTM", "BERT", "DistilBERT"])

    col1, col2, col3 = st.columns(3)

    if col1.button("Classifier"):
        start = time.time()
        # Appel fonction classification
        st.write(f"Thématique : **Business**")
        st.caption(f"Temps : {round(time.time()-start, 2)}s")

    if col2.button("Extraire Infos"):
        ents, _ = extraire_infos(texte)
        st.write("Entités détectées :", ents)

    if col3.button("Résumer"):
        with st.spinner('Génération du résumé...'):
            st.subheader("Résumé Abstractif (BART)")
            st.write(resume_abstractif(texte))